**Tabajo Practico 1 - Pandas & Spark RDD**

Todos los ejercicios se realizan utilizando el API de RDDs de Spark.

Se realizó todos los ejercicios en un solo notebook para no descargar varias veces la librería pyspark, validacion de credenciales Google.

Cada ejercicio se considera independiente uno del otro, no hay variables que se sobreescriban entre ejercicios, se crean nuevas.

# Instalamos e importamos librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Autenticamos con Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Bajamos los archivos csv

In [ ]:
downloaded = drive.CreateFile({'id':"1fYyEYw0JCjk6k94bO_GANtS83DofQGp1"})   # replace the id with id of file you want to access
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
downloaded = drive.CreateFile({'id':"1ViZVfHBZe84ZMWZ0nGUx8eL2gUNZ2hwg"})   # replace the id with id of file you want to access
downloaded.GetContentFile('googleplaystore_user_reviews.csv')

# Creamos el Spark Context

In [ ]:
# Crear una instancia de SparkSession y SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Leemos CSV

In [ ]:
# Leer los archivos CSV.
df_apps = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("googleplaystore.csv", header=True, inferSchema=True)
df_reviews = sqlContext.read.option("delimiter", ",").option("escape", '"').csv("googleplaystore_user_reviews.csv", header=True, inferSchema=True)

# Crear los RDDs a partir de los DataFrames
rdd_apps = df_apps.rdd
rdd_reviews = df_reviews.rdd

# Inspeccion

Analizamos el tipo de dato.

In [ ]:
# Mostras los tipos de datos de los RDDs
df_apps.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



In [ ]:
# Mostras los tipos de datos de los RDDs
df_reviews.printSchema()

root
 |-- App: string (nullable = true)
 |-- Translated_Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)
 |-- Sentiment_Polarity: string (nullable = true)
 |-- Sentiment_Subjectivity: string (nullable = true)



## Preprocesamiento de datos

Buscamos en la columna "Reviews" el dato mal cargado. Esta columna solo acepta números enteros positivos.

Para eso, filtramos los registros que no sean numéricos y los mostramos

In [ ]:
rdd_apps.filter(lambda x: not x["Reviews"].isdecimal()).take(2)


[Row(App='Life Made WI-Fi Touchscreen Photo Frame', Category='1.9', Rating=19.0, Reviews='3.0M', Size='1,000+', Installs='Free', Type='0', Price='Everyone', Content Rating=None, Genres='February 11, 2018', Last Updated='1.0.19', Current Ver='4.0 and up', Android Ver=None)]

Se considera que toda la fila esta mal cargada y se elimina.

In [ ]:
rdd_apps = rdd_apps.filter(lambda x: x["Reviews"] != "3.0M")

# Spark (⭐)

## 2) Teniendo en cuenta las reviews que reciben las aplicaciones, devolver una (al azar) de la aplicación que haya recibido la mayor cantidad de reviews positivas. (⭐)

- Filtrar solo reseñas positivas.

In [ ]:
positive_reviews_rdd = rdd_reviews.filter(lambda x: x['Sentiment'] == 'Positive')

- Filtrar solo valores numéricos en la columna "Reviews", por que la olumna se la tomo como string.

In [ ]:
rdd_apps_numeric_reviews = rdd_apps.filter(lambda x: x['Reviews'].isdecimal())

- Convertir la columna "Reviews" a un tipo numérico

In [ ]:
rdd_apps_numeric_reviews = rdd_apps_numeric_reviews.map(lambda x: (x['App'], int(x['Reviews'])))

- Obtener la aplicación con la mayor cantidad de reseñas positivas y su número de reseñas

In [ ]:
app_with_most_positive_reviews = rdd_apps_numeric_reviews.reduce(lambda a, b: a if a[1] >= b[1] else b)

- Filtrar todas las reseñas de la aplicación con la mayor cantidad de reseñas positivas

In [ ]:
reviews_of_app_with_most_positive_reviews = positive_reviews_rdd.filter(lambda x: x['App'] == app_with_most_positive_reviews[0])

- Seleccionar una reseña al azar de esa aplicación sin usar collect()

In [ ]:
random_review = reviews_of_app_with_most_positive_reviews.takeSample(False, 1)

In [ ]:
print("Aplicación con más reseñas positivas:", app_with_most_positive_reviews[0])
print("Reseña seleccionada al azar:")
print("Translated_Review:", random_review[0]['Translated_Review'])
print("Sentiment:", random_review[0]['Sentiment'])

Aplicación con más reseñas positivas: Facebook
Reseña seleccionada al azar:
Translated_Review: Why can't guys get together, guys still give problem refreshing newsfeed me. I'll literally middle watching something boom, refreshed, like Twitter I'm taking business back them, like years. Another thing, every video lags, Wi-Fi, Instagram, Twitter. Nothing else this, guys suck.
Sentiment: Positive


## 31) ¿Cuál es la app con mayor cantidad de instalaciones? (⭐)

La columna "Installs" se tomo como strin, tiene valores como "1,000,000+", "100,000,000+", "0+". Se limpian los datos y se convierten a números.

- Convertir la columna "Installs" a un tipo numérico.

In [ ]:
rdd_apps_installs = rdd_apps.map(lambda x: (x['App'], int(x['Installs'].replace('+', '').replace(',', ''))))

- Obtener la aplicación con la mayor cantidad de instalaciones y su número de instalaciones

In [ ]:
app_with_most_installs = rdd_apps_installs.reduce(lambda a, b: a if a[1] >= b[1] else b)

In [ ]:
print("Aplicación con más instalaciones:", app_with_most_installs[0])

Aplicación con más instalaciones: Google Play Books


# Spark (⭐⭐)

## 10) Cual es la categoría con mayor promedio de apps que hayan sido al menos 1 vez calificadas como positivas (⭐⭐)

Ejemplo con números hipotéticos:

Supongamos que en la categoría "Juegos" hay 100 aplicaciones únicas.

Luego encontramos que de esas 100 aplicaciones, 60 tienen al menos 1 reseña positiva.

Entonces los cálculos serían:

- Total de apps únicas en la categoría: 100
- Apps con al menos 1 reseña positiva: 60
- Porcentaje de apps positivas: 60/100 = 0.6 (60%)

Así que para la categoría "Juegos" el porcentaje de apps con al menos una reseña positiva es del 60%.

Este mismo cálculo se hace para cada categoría:

- Contar el total de apps únicas en esa categoría
- Contar cuántas tienen al menos 1 reseña positiva
- Dividir las positivas sobre el total para obtener el porcentaje

In [ ]:
rdd_positive_reviews = rdd_reviews.filter(lambda x: x["Sentiment"] == "Positive")

In [ ]:
rdd_positive_counts = rdd_positive_reviews.map(lambda x: (x["App"], 1))
rdd_app_positive_counts = rdd_positive_counts.reduceByKey(lambda a, b: a)

In [ ]:
rdd_app_category_join = rdd_apps.map(lambda x: (x["App"], x["Category"])).distinct()

- Calcular el total de aplicaciones en cada categoría

In [ ]:
rdd_category_counts = rdd_app_category_join.map(lambda x: (x[1], 1)).reduceByKey(lambda a, b: a + b)
rdd_result = rdd_app_category_join.join(rdd_app_positive_counts)

- Calcular el total de aplicaciones con al menos una reseña positiva en cada categoría

In [ ]:
rdd_category_positive_counts = rdd_result.map(lambda x: (x[1][0], 1)).reduceByKey(lambda a, b: a + b)

- Calcular los porcentajes de aplicaciones con al menos una reseña positiva en cada categoría

In [ ]:
rdd_category_positive_percentages = rdd_category_positive_counts.join(rdd_category_counts).map(lambda x: (x[0], x[1][0] / x[1][1]))

- Encontrar la categoría con el mayor porcentaje de aplicaciones positivas -> (Category, promedio)

In [ ]:
rdd_category_positive_percentages.reduce(lambda a, b: a if a[1] >= b[1] else b)

('ENTERTAINMENT', 0.24509803921568626)

## 18) Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo. (⭐⭐)

La columna "Android Ver" y la columna "Size", al cargarla se la tomo como string. La columna "Size" que tiene valores: "Varies with device", "1.1M" y "175k"

Para los cálculos se usará la unidad de medida MB.

- Filtramos en la columna "Android Ver" los valores que no sean "Varies with device" y los que no sean "NaN"

In [ ]:
rdd_apps_android_ver = rdd_apps.filter(lambda x: x["Android Ver"] != "Varies with device" and x["Android Ver"] != "NaN")

- Filtramos en la columna "Size" los valores que no sean "Varies with device" y los que no sean "NaN"

In [ ]:
rdd_apps_android_size = rdd_apps_android_ver.filter(lambda x: x["Size"] != "Varies with device" and x["Size"] != "NaN")

- Función que elimina la letra "M" y "k" y lo convierte a float con la unidad de medida en Mega.

In [ ]:
def convert_to_mega(value):
    if isinstance(value, str):
        if 'M' in value:
            return float(value.replace('M', ''))
        elif 'k' in value:
            return float(value.replace('k', '')) / 1000
    return float(value) / 1000000

- Convertimos la columna "Size" a un tipo numérico en la unidad de medida MB.

In [ ]:
rdd_apps_android_size = rdd_apps_android_size.map(lambda x: (x['Android Ver'], convert_to_mega(x['Size'])))

- Calculamos el promedio de tamaño de las aplicaciones por versión de Android

In [ ]:
rdd_apps_android_size = rdd_apps_android_size.map(lambda x: (x[0], (x[1], 1))).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])).map(lambda x: (x[0], x[1][0] / x[1][1]))

- Cantidad

In [ ]:
rdd_apps_android_size.count()

32

- Mostramos-> (Android Ver, Promedio en Megas)

In [ ]:
cantidad = 32
rdd_apps_android_size.take(cantidad)

[('4.0.3 and up', 22.081509291121826),
 ('4.2 and up', 22.1400940860215),
 ('4.4 and up', 25.819297802197795),
 ('2.3 and up', 20.809874015748047),
 ('3.0 and up', 14.699755186721982),
 ('4.1 and up', 26.02670781182566),
 ('4.0 and up', 17.10488304959288),
 ('2.2 and up', 8.499759336099585),
 ('5.0 and up', 24.666352517985594),
 ('6.0 and up', 22.50761403508772),
 ('1.6 and up', 3.1191379310344827),
 ('2.1 and up', 5.6054626865671695),
 ('5.1 and up', 23.622304347826088),
 ('1.5 and up', 5.071631578947368),
 ('7.0 and up', 24.314650000000004),
 ('4.3 and up', 22.603359307359316),
 ('4.0.3 - 7.1.1', 6.6),
 ('2.0 and up', 6.256806451612904),
 ('2.3.3 and up', 19.221510869565225),
 ('3.2 and up', 9.860638888888888),
 ('4.4W and up', 13.1),
 ('7.1 and up', 32.00425),
 ('7.0 - 7.1.1', 7.1),
 ('8.0 and up', 20.083333333333332),
 ('5.0 - 8.0', 23.8),
 ('3.1 and up', 13.413799999999998),
 ('2.0.1 and up', 22.22214285714286),
 ('4.1 - 7.1.1', 7.9),
 ('5.0 - 6.0', 11.0),
 ('1.0 and up', 3.8555),

## 19) Para cada categoría, indicar cuál es la aplicación que tiene mayor cantidad de reviews con sentimiento negativo (⭐⭐)

- Filtrar las review con sentimiento negativo.

In [ ]:
rdd_negative_reviews = rdd_reviews.filter(lambda x: x["Sentiment"] == "Negative")

- Crea un nuevo RDD (App, cantidad de revisiones negativas)

In [ ]:
rdd_negative_reviews_count = rdd_negative_reviews.map(lambda x: (x['App'], 1)).reduceByKey(lambda a, b: a + b)

- Crea un RDD que contenga tuplas de la forma (App, (Category, cantidad de revisiones negativas))

In [ ]:
category_max_negative_reviews = rdd_apps.map(lambda x: (x['App'], x['Category'])).distinct().join(rdd_negative_reviews_count)

- Encuentra la aplicación con la mayor cantidad de review negativas por categoría -> (Catagory,(App, cantidad de revisiones negativas))

In [ ]:
categoty_app_reviews = category_max_negative_reviews.map(lambda x: (x[1][0], (x[0], x[1][1]))).reduceByKey(lambda a, b: a if a[1] >= b[1] else b)

- Cantidad de categorias

In [ ]:
categoty_app_reviews.count()

33

- Mostramos las 33 categorias -> (Catagory,(App, cantidad de revisiones negativas))

In [ ]:
number_of_categories = 33
categoty_app_reviews.take(number_of_categories)

[('COMMUNICATION', ('Azar', 25)),
 ('ENTERTAINMENT', ('Colorfy: Coloring Book for Adults - Free', 45)),
 ('FINANCE', ('Google Pay', 30)),
 ('MEDICAL', ('Anthem Anywhere', 28)),
 ('SOCIAL', ('Facebook', 60)),
 ('TOOLS', ('Gboard - the Google Keyboard', 34)),
 ('NEWS_AND_MAGAZINES',
  ('Fox News – Breaking News, Live Video & News Alerts', 42)),
 ('EVENTS', ('DroidAdmin for Android - Advice', 12)),
 ('BEAUTY', ('BestCam Selfie-selfie, beauty camera, photo editor', 32)),
 ('COMICS', ('DC Comics', 1)),
 ('EDUCATION', ('Duolingo: Learn Languages Free', 34)),
 ('FAMILY', ('Candy Crush Saga', 126)),
 ('LIBRARIES_AND_DEMO', ('Aviary Stickers: Free Pack', 12)),
 ('LIFESTYLE', ('Fashion in Vogue', 32)),
 ('PHOTOGRAPHY', ('Google Photos', 47)),
 ('PARENTING', ('Baby Name Together', 6)),
 ('MAPS_AND_NAVIGATION', ('FindShip', 9)),
 ('ART_AND_DESIGN', ('Coloring book moana', 14)),
 ('AUTO_AND_VEHICLES', ('AutoScout24 Switzerland – Find your new car', 3)),
 ('BOOKS_AND_REFERENCE', ('Amazon Kindle', 33